# **Lasso**

*Fonctionne avec Python 3.10.9 (Anaconda 23.3.1)*

## **Importation des bibliothèques**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from sklearn.linear_model import Lasso

# Pour gridsearch
from sklearn import model_selection

from sklearn import metrics

## **Importation du dataset**

In [2]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)

## **Fonctions**

In [3]:
def split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification):

    ##############################
    # Split du dataset en X et y #
    ##############################

    global X
    global y

    X = data.loc[:,features_list] # On ajoute nos features dans le X
    y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y

    ############################
    # Scaling et Encoding de X #
    ############################

    global feature_encoder

    numeric_transformer = StandardScaler()

    categorical_transformer = OneHotEncoder()

    feature_encoder = ColumnTransformer(
                transformers=[
                    ('cat', categorical_transformer, categorical_features),    
                    ('num', numeric_transformer, numeric_features)
                    ]
                )
    X = feature_encoder.fit_transform(X)


    ####################################
    # Split de X et y en train et test #
    ####################################

    global X_train
    global X_test
    global y_train
    global y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0,
                                                        stratify=eval(stratification))

## **Features**

In [4]:
######################
# Choix des features #
######################

dataset_name = data

# Si rien mettre []
categorical_features = [
                        'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'YearBuilt'
                       ]

# Si rien mettre []
numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)'
                   ]

# Toutes les features
features_list = categorical_features + numeric_features

## ***Lasso***

In [5]:
# Targets à estimer
targets = ['SiteEnergyUseWN_kBtu', 'TotalGHGEmissions']

# "y" pour stratifier y, sinon "None"
stratification = "None"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========\n")


    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)


    # Entraînement simple
    modele = Lasso() # Paramètres par défaut
    modele.fit(X_train, y_train) # Étape d'entraînement
    print("--- Entraînement simple : ---")
    print(f"Le score R² du modèle pour prédire {i} est de {modele.score(X_train, y_train):.3f} (train) et {modele.score(X_test, y_test):.3f} (test).")


    ###############################################
    # Paramétrage de GridSearchCV et entraînement #
    ###############################################

    print("\n--- GridSearch : ---")

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = {'alpha':[0.5, 1, 2, 3, 3.5, 4, 5, 10, 100, 1000, 10000]} #, 'max_iter':[50, 100, 1000]

    # Déterminer le score qu'on veut optimiser
    score = 'r2'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        Lasso(fit_intercept=False), # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement ({i}): {grid.best_params_}")

    # Afficher les performances correspondantes
    if details == True:
        print('Résultats pour chaque fold :')
        for j in range(cv):
            print(f"Fold n°{j+1} :", eval(f"grid.cv_results_['split{j}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

        print("\nRésultats de la validation croisée :")
        for mean, std, params in zip(
                grid.cv_results_['mean_test_score'], # score moyen
                grid.cv_results_['std_test_score'],  # écart-type du score
                grid.cv_results_['params']           # valeur de l'hyperparamètre
            ):

            print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")


    ###################################################################
    # Prédiction sur le jeu de test avec les hyperparamètres optimaux #
    ###################################################################

    # GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test ({i}, avec paramètres optimaux): {metrics.r2_score(y_test, y_pred):.3f}")

    print('_'*80, "\n")

    

=========== [SiteEnergyUseWN_kBtu] ===========



C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6000116051668396.0, tolerance: 15178421184405.035
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.563834142108177e+16, tolerance: 17342661880521.652
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6668611545561566e+16, tolerance: 16321093971582.63
  model = cd_fast.sparse_enet_coordinate_descent

--- Entraînement simple : ---
Le score R² du modèle pour prédire SiteEnergyUseWN_kBtu est de 0.695 (train) et 0.642 (test).

--- GridSearch : ---


C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.416881504689826e+16, tolerance: 16022158966741.512
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.832831602170268e+16, tolerance: 17703673230770.49
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6976730528101456e+16, tolerance: 15998628335024.346
  model = cd_fast.sparse_enet_coordinate_desc

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (SiteEnergyUseWN_kBtu): {'alpha': 10000}
Score sur le jeu de test (SiteEnergyUseWN_kBtu, avec paramètres optimaux): 0.652
________________________________________________________________________________ 

=========== [TotalGHGEmissions] ===========

--- Entraînement simple : ---
Le score R² du modèle pour prédire TotalGHGEmissions est de 0.466 (train) et 0.352 (test).

--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (TotalGHGEmissions): {'alpha': 3}
Score sur le jeu de test (TotalGHGEmissions, avec paramètres optimaux): 0.269
________________________________________________________________________________ 

